© 2024 Rebecca J. Rousseau and Justin B. Kinney, *Algebraic and diagrammatic methods for the rule-based modeling of multi-particle complexes*. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).
___

# Stochastic simulation for heterodimer system

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pdb
from gillespie import get_operator_list, get_factory_list, Trajectory, Recorder

In [2]:
### Define model parameters and simulation parameters

# Define model parameters

N = 100 # Number of indices
creation_rate_A = 2
annihilation_rate_A = 1
creation_rate_B = 1
annihilation_rate_B = 1
bonding_rate = 0.03
unbonding_rate = 1

# Define simulation parameters

num_steps=1200
record_every=10
fields_to_record=['A','B','a','b','I']
complexes_to_record=['A monomer','B monomer', 'AB dimer']

In [3]:
### Define op_list, factory_list, and get_complex_name() function

# Define index values

A_indices = [i for i in range(N)]
B_indices = [j for j in range(N,2*N)]
I_indices = [(i,j) for i in A_indices for j in B_indices]

# Define operator list

op_list = get_operator_list("A", A_indices) + \
          get_operator_list("a", A_indices) + \
          get_operator_list("B", B_indices) + \
          get_operator_list("b", B_indices) + \
          get_operator_list("I", I_indices)

# Define factory list

factory_list = \
    get_factory_list(op_list=op_list,
                    rate=creation_rate_A,
                    spec='A_i_hat a_i_tilde',
                    indices=A_indices) + \
    get_factory_list(op_list=op_list,
                    rate=annihilation_rate_A,
                    spec='A_i_check a_i_tilde',
                    indices=A_indices) + \
    get_factory_list(op_list=op_list,
                    rate=creation_rate_B,
                    spec='B_i_hat b_i_tilde',
                    indices=B_indices) + \
    get_factory_list(op_list=op_list,
                    rate=annihilation_rate_B,
                    spec='B_i_check b_i_tilde',
                    indices=B_indices) + \
    get_factory_list(op_list=op_list,
                    rate=bonding_rate,
                    spec='A_i_bar B_j_bar a_i_hat b_j_hat I_ij_hat',
                    indices=I_indices) + \
    get_factory_list(op_list=op_list,
                    rate=unbonding_rate,
                    spec='A_i_bar B_j_bar a_i_check b_j_check I_ij_check',
                    indices=I_indices)

# Define function that, given a list of operators, returns a dictionary of counts for assembled complexes

def get_complex_name(op_list):
    num_A = sum([1 for op in op_list if op.name=='A'])
    num_B = sum([1 for op in op_list if op.name=='B'])
    num_I = sum([1 for op in op_list if op.name=='I'])
    if num_A==1 and num_B==0:
        complex_name='A monomer'
    elif num_A==0 and num_B==1:
        complex_name='B monomer'
    elif num_A==1 and num_B==1:
        complex_name='AB dimer'
    else:
        pdb.set_trace()
        assert False, 'Cannot identify complex'
    return complex_name

In [4]:
### Run simulation (model-agnostic code)

# Create Recorder object to store results

cycles = 500
tsteps = int(num_steps/record_every)
allcycfields = np.empty((len(fields_to_record)+1,tsteps,cycles))
allcyccomplexes = np.empty((len(complexes_to_record)+1,tsteps,cycles))
for i in range(cycles):
    recorder = Recorder(num_steps=num_steps,
                    record_every=record_every,
                    fields_to_record=fields_to_record,
                    complexes_to_record=complexes_to_record,
                    get_complex_name_fn=get_complex_name)

    # Create and run Trajectory object to compute trajectory (model-agnostic)
    trajectory = Trajectory(factory_list=factory_list)
    trajectory.run(recorder=recorder, verbose=True)
    
    df = recorder.field_counts_df.copy()
    dc = recorder.complex_counts_df.copy()
    for ind, column in enumerate(df.columns):
        allcycfields[ind,:,i] = df[str(column)].to_numpy()
    for ind, column in enumerate(dc.columns):
        allcyccomplexes[ind,:,i] = dc[str(column)].to_numpy()

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [5]:
# METHOD for averaging over sampled trajectories: LINEAR INTERPOLATION on each trajectory, then average

Nf = 5001
xt = np.linspace(0,10,num=Nf)
avgtablecomp = np.empty((Nf,len(complexes_to_record)+1))
avgtablecomp[:,0] = np.transpose(xt)
stdtablecomp = np.empty((Nf,len(complexes_to_record)+1))
stdtablecomp[:,0] = np.transpose(np.zeros(Nf))
for i in range(1,len(complexes_to_record)+1):
    tempinterp = np.empty((Nf,cycles))
    for j in range(cycles):
        xd = np.transpose(np.concatenate([[0],allcyccomplexes[0,:,j]]))
        yd = np.transpose(np.concatenate([[0],allcyccomplexes[i,:,j]]))
        tempinterp[:,j] = np.transpose(np.interp(xt,xd,yd))
    avgtablecomp[:,i] = np.mean(tempinterp,axis=1)
    stdtablecomp[:,i] = np.std(tempinterp,axis=1)

In [6]:
# Save time bins, average counts, and standard deviations

folderpath = "../simulationdata"
np.savetxt(f"{folderpath}/heterodimer_timebins_N{N}.csv",xt,delimiter=",")
np.savetxt(f"{folderpath}/heterodimer_Amonbins_N{N}.csv",avgtablecomp[:,1],delimiter=",")
np.savetxt(f"{folderpath}/heterodimer_Amonbinsstd_N{N}.csv",stdtablecomp[:,1],delimiter=",")
np.savetxt(f"{folderpath}/heterodimer_Bmonbins_N{N}.csv",avgtablecomp[:,2],delimiter=",")
np.savetxt(f"{folderpath}/heterodimer_Bmonbinsstd_N{N}.csv",stdtablecomp[:,2],delimiter=",")
np.savetxt(f"{folderpath}/heterodimer_ABdimbins_N{N}.csv",avgtablecomp[:,3],delimiter=",")
np.savetxt(f"{folderpath}/heterodimer_ABdimbinsstd_N{N}.csv",stdtablecomp[:,3],delimiter=",")